In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults, ARMA
from statsmodels.tsa.arima_process import ArmaProcess
from sklearn.metrics import mean_squared_error
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from timeseries_functions import index_to_datetime, plot_all_df_columns, weekly_resample, plot_series,\
plot_series_save_fig, plot_series_and_differences, run_augmented_Dickey_Fuller_test, \
plot_autocorrelation, plot_partial_autocorrelation, plot_decomposition

In [ ]:
from timeseries_functions import make_col_vector, make_design_matrix, fit_linear_trend,\
plot_trend_data, plot_linear_trend

In [ ]:
from AR_MA_functions import get_AR_model, plot_AR_model, get_AR_model_order_BIC,\
plot_BIC_AR_model, get_MA_model, plot_MA_model

In [ ]:
from ARIMA_functions import get_ARIMA_model, plot_ARIMA_model, plot_ARIMA_resids,\
get_ARIMA_forecast, plot_ARIMA_forecast_and_CI, plot_data_plus_ARIMA_predictions, \
test_rolling_ARIMA_forecast,get_predictions_df_and_plot_rolling_ARIMA_forecast

In [ ]:
from ARIMA_functions import get_ARIMAX_model, plot_ARIMAX_model, plot_ARIMAX_model_save_fig, \
plot_data_plus_ARIMAX_predictions, get_ARIMAX_predictions, get_ARIMAX_training_MSE,\
get_ARIMAX_train_test_predictions, get_ARIMAX_train_test_MSE

#### import data

In [ ]:
appointments = pd.read_csv('./data/appointments_through_04-2018.csv', index_col=0)

In [ ]:
appointments.index = pd.to_datetime(appointments.index)

In [ ]:
appointments['Hours_Spent'] = appointments['AppointmentDuration'] / 60

In [ ]:
appointments.head()

In [ ]:
# group by specialty
doctor = appointments[appointments['Specialty'] == 'doctor']
RN_PA = appointments[appointments['Specialty'] == 'RN/PA']
therapist = appointments[appointments['Specialty'] == 'therapist']

In [ ]:
doctor.columns

In [ ]:
# Sum Appointment hours by date
dr_hours = doctor.groupby(doctor.index.date)['Hours_Spent'].sum()
RNPA_hours = RN_PA.groupby(RN_PA.index.date)['Hours_Spent'].sum()
therapist_hours = therapist.groupby(therapist.index.date)['Hours_Spent'].sum()

In [ ]:
dr_hours.head(), dr_hours.tail()

In [ ]:
# import weekly numbers data
num_dr = pd.read_csv('./data/doctors_hours_per_provider.csv', index_col=0)
num_RNPA = pd.read_csv('./data/RNPA_hours_per_provider.csv',  index_col=0)
num_therapists = pd.read_csv('./data/therapists_hours_per_provider.csv',  index_col=0)

In [ ]:
providers = [num_dr, num_RNPA, num_therapists]
hours = [dr_hours, RNPA_hours, therapist_hours]

#### convert index to datetime and downsample hours data for each series to weekly freq

In [ ]:
for p in providers:
    p.index = pd.to_datetime(p.index)
for h in hours:
    h.index = pd.to_datetime(h.index)

In [ ]:
# resample to weekly data
dr_hours = weekly_resample(dr_hours)
RNPA_hours = weekly_resample(RNPA_hours)
therapist_hours = weekly_resample(therapist_hours)

In [ ]:
# drop first row --> partial week 
dr_hours = dr_hours[1:]
RNPA_hours = RNPA_hours[1:]
therapist_hours = therapist_hours[1:]

In [ ]:
dr_hours.tail()

In [ ]:
providers = [num_dr, num_RNPA, num_therapists]
hours = [dr_hours, RNPA_hours, therapist_hours]

In [ ]:
# name each series
num_dr.name = 'Number of Doctors'
num_RNPA.name = 'Number of RN/PAs'
num_therapists.name = 'Number of Therapists'

In [ ]:
dr_hours.name = 'Doctors Hours'
RNPA_hours.name = 'RN/PAs Hours'
therapist_hours.name = 'Therapists Hours'

#### test for stationarity in all time series

In [ ]:
providers = [num_dr, num_RNPA, num_therapists]
hours = [dr_hours, RNPA_hours, therapist_hours]

In [ ]:
for p in providers:  
    print(p.name)
    run_augmented_Dickey_Fuller_test(series=p['Number_Providers'], num_diffs=2)

In [ ]:
for h in hours:
    print(h.name)
    run_augmented_Dickey_Fuller_test(series=h, num_diffs=2)

#### determine number of AR and MA terms to add for each time series

In [ ]:
# Plot ACF and PACF of first difference of each series
for p in providers:
    params = {'figure.figsize': [4,3],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
    plot_partial_autocorrelation(series=p['Number_Providers'], params=params, lags=30, alpha=0.05, title='PACF {}'.format(p.name))
    plot_autocorrelation(series=p['Number_Providers'], params=params, lags=30, alpha=0.05, title='ACF {}'.format(p.name))

In [ ]:
for h in hours:
    params = {'figure.figsize': [4,3],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
    plot_partial_autocorrelation(series=h, params=params, lags=30, alpha=0.05, title='PACF {}'.format(h.name))
    plot_autocorrelation(series=h, params=params, lags=30, alpha=0.05, title='ACF {}'.format(h.name))

#### get ARIMAX model using num_provider as exogenous variable and plot

#### Doctors

In [ ]:
start_date = '2015-01-19' # b/c start data differenced away
end_date = '2018-04-30'
end_pred = '2018-09-30'

In [ ]:
dr_results, dr_summary, dr_params, dr_residuals = get_ARIMAX_model(data=dr_hours,\
                    exog_var=num_dr, order=(5,1,0))

In [ ]:
num_dr

In [ ]:
def plot_ARIMAX_model_save_fig(data, order, exog_var, start, end, title='', xlabel='', ylabel='',
                              figname='figure.png'):
    """Plots ARIMAX model
    data: pandas Series
    order: (p,d,q) format
    exog_var = exogenous variable as pandas Series
    start/end: starting/ending dates for plot (x_axis)
    """
    results = ARIMA(endog=data, order=order, exog=exog_var).fit()
    fig = results.plot_predict(start=start, end=end,exog=exog_var)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.show()
    fig.savefig(figname)

In [ ]:
# plot and save doctors figure using 3 MA components
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

plot_ARIMAX_model_save_fig(data=dr_hours, order=(5,1,3), exog_var=num_dr, start=start_date,\
                end=end_pred, title='Doctors', xlabel='Time', ylabel='Appointment Hours',
                              figname='./images/dr_AR[5]IMA[3]X.png')

In [ ]:
# plot and save doctors figure using 2 MA components
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

plot_ARIMAX_model_save_fig(data=dr_hours, order=(5,1,0), exog_var=num_dr, start=start_date,\
                end=end_pred, title='Doctors', xlabel='Time', ylabel='Appointment Hours',
                              figname='./images/dr_AR[5]IMA[0]X.png')

In [ ]:
# using 5 AR components test 0-3 MA components
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

for i in range(4):
    fig = plot_ARIMAX_model(data=dr_hours, order=(5,1,i), exog_var=num_dr, start=start_date,\
                         end=end_pred, title='AR5/MA{i}'.format(i=i))

In [ ]:
# getARIMAX predictions and data+predictions dataframe for doctors
start_date = '2015-01-19'
end_pred = '2018-09-30'
dr_forecast, dr_data_plus_forecast = get_ARIMAX_predictions(data=dr_hours, order=(5,1,0), start=start_date,\
                    end=end_pred, exog=num_dr, typ='levels')

In [ ]:
dr_forecast, dr_data_plus_forecast = get_ARIMAX_predictions(data=dr_hours, order=(5,1,3), start=start_date,\
                    end=end_pred, exog=num_dr, typ='levels')

In [ ]:
dr_forecast

In [ ]:
dr_data_plus_forecast

In [ ]:
dr_data_plus_forecast.columns

In [ ]:
# rename columns in forecast df
dr_data_plus_forecast.columns = ['Appointment_Hours', 'Predicted_Hours']

In [ ]:
dr_data_plus_forecast

In [ ]:
num_dr.columns

In [ ]:
# Round Predicted Hours values
dr_data_plus_forecast['Predicted_Hours'] = round(dr_data_plus_forecast['Predicted_Hours'], 2)

In [ ]:
dr_data_plus_forecast

In [ ]:
dr_data_plus_forecast['Number_Providers'] = num_dr['Number_Providers']

In [ ]:
# add number of providers and hours/provider columns
dr_data_plus_forecast['Pred_Hours_per_Provider'] = round(dr_data_plus_forecast['Predicted_Hours']/dr_data_plus_forecast['Number_Providers'], 2)

In [ ]:
dr_data_plus_forecast

In [ ]:
dr_data = dr_data_plus_forecast['2015-01-12':'2018-04-30']

In [ ]:
dr_data_plus_forecast['Hours_per_Provider'] = dr_data_plus_forecast['Appointment_Hours'] / dr_data_plus_forecast['Number_Providers']

In [ ]:
avg_dr_hours = dr_data_plus_forecast['Hours_per_Provider'].mean()

In [ ]:
avg_dr_hours

In [ ]:
# get predicted number of providers rounded up
dr_data_plus_forecast['Predicted_Num_Providers'] = round(dr_data_plus_forecast['Predicted_Hours'] / avg_dr_hours, 1)

In [ ]:
dr_data_plus_forecast

In [ ]:
# output full forecast dataframe to csv
dr_data_plus_forecast.to_csv('./data/dr_arimax_forecast.csv')

In [ ]:
# get 12 week forecast
dr_12wk_arimax = dr_data_plus_forecast['2018-05-07':'2018-07-30'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
dr_12wk_arimax.index = dr_12wk_arimax.index.date

In [ ]:
# get 8-16 week forecast
dr_8_to_16wk_arimax = dr_data_plus_forecast['2018-06-25':'2018-08-26'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
dr_8_to_16wk_arimax.index = dr_8_to_16wk_arimax.index.date

In [ ]:
dr_8_to_16wk_arimax

In [ ]:
from pandas.tools.plotting import table
fig, ax = plt.subplots(figsize=(10,10)) # set size frame
ax.xaxis.set_visible(False)  # hide axes
ax.yaxis.set_visible(False)  
ax.set_frame_on(False)  
tabla = table(ax, dr_8_to_16wk_arimax, loc='center', colWidths=[0.3]*len(dr_8_to_16wk_arimax.columns))
tabla.auto_set_font_size(False) 
tabla.set_fontsize(20)
tabla.scale(1.5, 2) 
plt.savefig('./images/dr_8_to_16wk_arimax.png', transparent=True)

In [ ]:
dr_12wk_arimax

In [ ]:
# plot acf/pacf of Dr ARIMAX residuals
params = {'figure.figsize': [4,4],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_partial_autocorrelation(series=dr_residuals, params=params, lags=30, alpha=0.05, title='PACF {}'.format('Dr ARIMAX residuals'))
plot_autocorrelation(series=dr_residuals, params=params, lags=30, alpha=0.05, title='ACF {}'.format('Dr ARIMAX residuals'))

#### RN/PAs

In [ ]:
start_date = '2015-01-19' # b/c start data differenced away
end_date = '2018-04-30'
end_pred = '2018-09-30'
RNPA_order = (4,1,0)

In [ ]:
RNPA_results, RNPA_summary, RNPA_params, RNPA_residuals = get_ARIMAX_model(data=dr_hours,\
                    exog_var=num_dr, order=RNPA_order)

In [ ]:
# plot and save RN/PA figure 
# using 4 AR components and 0 MA components
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

plot_ARIMAX_model_save_fig(data=dr_hours, order=(4,1,0), exog_var=num_RNPA, start=start_date,\
                end=end_pred, title='Doctors', xlabel='Time', ylabel='Appointment Hours',
                              figname='./images/RNPA_ARIMAX.png')

In [ ]:
# using 4 AR components test 0-3 MA components
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

for i in range(4):
    fig = plot_ARIMAX_model(data=RNPA_hours, order=(4,1,i), exog_var=num_RNPA, start=start_date,\
                         end=end_pred, title='AR4/MA{i}'.format(i=i))

In [ ]:
# getARIMAX predictions and data+predictions dataframe for RN/PAs
start_date = '2015-01-19'
end_pred = '2018-09-30'
RNPA_forecast, RNPA_data_plus_forecast = get_ARIMAX_predictions(data=RNPA_hours, order=(4,1,0), start=start_date,\
                    end=end_pred, exog=num_RNPA, typ='levels')

In [ ]:
RNPA_forecast

In [ ]:
# rename columns in forecast df
RNPA_data_plus_forecast.columns = ['Appointment_Hours', 'Predicted_Hours']

# Round Predicted Hours values
RNPA_data_plus_forecast['Predicted_Hours'] = round(RNPA_data_plus_forecast['Predicted_Hours'], 2)

# add number of providers and hours/provider columns
RNPA_data_plus_forecast['Number_Providers'] = num_RNPA['Number_Providers']
RNPA_data_plus_forecast['Pred_Hours_per_Provider'] = round(RNPA_data_plus_forecast['Predicted_Hours']/RNPA_data_plus_forecast['Number_Providers'], 2)

In [ ]:
avg_RNPA_hours = RNPA_data_plus_forecast['Pred_Hours_per_Provider'].mean()

In [ ]:
avg_RNPA_hours

In [ ]:
# get predicted number of providers rounded up
RNPA_data_plus_forecast['Predicted_Num_Providers'] = round(RNPA_data_plus_forecast['Predicted_Hours'] / avg_RNPA_hours, 2)

In [ ]:
RNPA_data_plus_forecast

In [ ]:
# output full forecast dataframe to csv
# RNPA_data_plus_forecast.to_csv('./data/RNPA_arimax_forecast.csv')

In [ ]:
# get 12 week forecast
RNPA_12wk_arimax = RNPA_data_plus_forecast['2018-05-07':'2018-07-30'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
RNPA_12wk_arimax.index = RNPA_12wk_arimax.index.date

In [ ]:
# get 8-16 week forecast
RNPA_8_to_16wk_arimax = RNPA_data_plus_forecast['2018-06-25':'2018-08-26'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
RNPA_8_to_16wk_arimax.index = RNPA_8_to_16wk_arimax.index.date

In [ ]:
from pandas.tools.plotting import table
fig, ax = plt.subplots(figsize=(10,10)) # set size frame
ax.xaxis.set_visible(False)  # hide axes
ax.yaxis.set_visible(False)  
ax.set_frame_on(False)  
tabla = table(ax, RNPA_8_to_16wk_arimax, loc='center', colWidths=[0.25]*len(RNPA_8_to_16wk_arimax.columns))
tabla.auto_set_font_size(False) 
tabla.set_fontsize(16)
tabla.scale(1.5, 2) 
plt.savefig('./images/RNPA_8_to_16wk_arimax.png', transparent=True)

In [ ]:
# plot acf/pacf of RN/PA ARIMAX residuals
params = {'figure.figsize': [4,4],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_partial_autocorrelation(series=RNPA_residuals, params=params, lags=30, alpha=0.05, title='PACF {}'.format('RN/PA ARIMAX residuals'))
plot_autocorrelation(series=RNPA_residuals, params=params, lags=30, alpha=0.05, title='ACF {}'.format('RN/PA ARIMAX residuals'))

#### therapists

In [ ]:
start_date = '2015-01-19' # b/c start data differenced away
end_date = '2018-04-30'
end_pred = '2018-09-30'
therapist_order = (5,1,0)

In [ ]:
therapist_results, therapist_summary, therapist_params, therapist_residuals = get_ARIMAX_model(data=therapist_hours,\
                    exog_var=num_therapists, order=therapist_order)

In [ ]:
# plot and save therapist figure 
# using 5 AR components and 0 MA components
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)

plot_ARIMAX_model_save_fig(data=therapist_hours, order=(5,1,0), exog_var=num_therapists, start=start_date,\
                end=end_pred, title='Doctors', xlabel='Time', ylabel='Appointment Hours',
                              figname='./images/therapist_ARIMAX.png')

In [ ]:
# getARIMAX predictions and data+predictions dataframe for therapists
start_date = '2015-01-19'
end_pred = '2018-09-30'
ther_forecast, ther_data_plus_forecast = get_ARIMAX_predictions(data=therapist_hours, order=(5,1,0), start=start_date,\
                    end=end_pred, exog=num_therapists, typ='levels')

In [ ]:
# rename columns in forecast df
ther_data_plus_forecast.columns = ['Appointment_Hours', 'Predicted_Hours']

# Round Predicted Hours values
ther_data_plus_forecast['Predicted_Hours'] = round(ther_data_plus_forecast['Predicted_Hours'])

# add number of providers and hours/provider columns
ther_data_plus_forecast['Number_Providers'] = num_therapists['Number_Providers']
ther_data_plus_forecast['Pred_Hours_per_Provider'] = round(ther_data_plus_forecast['Predicted_Hours']/ther_data_plus_forecast['Number_Providers'])

In [ ]:
avg_ther_hours = ther_data_plus_forecast['Pred_Hours_per_Provider'].mean()

In [ ]:
# get predicted number of providers rounded up
ther_data_plus_forecast['Predicted_Num_Providers'] = round(ther_data_plus_forecast['Predicted_Hours'] / avg_ther_hours)

In [ ]:
# output full forecast dataframe to csv
ther_data_plus_forecast.to_csv('./data/therapist_arimax_forecast.csv')

In [ ]:
# get 12 week forecast
ther_12wk_arimax = ther_data_plus_forecast['2018-05-07':'2018-07-30'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
ther_12wk_arimax.index = ther_12wk_arimax.index.date

In [ ]:
# get 8-16 week forecast
ther_8_to_16wk_arimax = ther_data_plus_forecast['2018-06-25':'2018-08-26'][['Predicted_Hours', 'Predicted_Num_Providers']]
# keep only date in index, drop time
ther_8_to_16wk_arimax.index = ther_8_to_16wk_arimax.index.date

In [ ]:
from pandas.tools.plotting import table
fig, ax = plt.subplots(figsize=(10,10)) # set size frame
ax.xaxis.set_visible(False)  # hide axes
ax.yaxis.set_visible(False)  
ax.set_frame_on(False)  
tabla = table(ax, ther_8_to_16wk_arimax, loc='center', colWidths=[0.25]*len(ther_8_to_16wk_arimax.columns))
tabla.auto_set_font_size(False) 
tabla.set_fontsize(16)
tabla.scale(1.5, 2) 
plt.savefig('./images/ther_8_to_16wk_arimax.png', transparent=True)

In [ ]:
# plot acf/pacf of therapist ARIMAX residuals
params = {'figure.figsize': [4,4],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plot_partial_autocorrelation(series=therapist_residuals, params=params, lags=30, alpha=0.05, title='PACF {}'.format('Therapist ARIMAX residuals'))
plot_autocorrelation(series=therapist_residuals, params=params, lags=30, alpha=0.05, title='ACF {}'.format('Therapist ARIMAX residuals'))

#### get data and forecast, save predictions to csv

#### doctors

In [ ]:
dr_forecast, dr_predictions = get_ARIMAX_predictions(data=dr_hours, order=(5,1,3),\
        start=start_date, end=end_pred, exog=num_dr, typ='levels')

In [ ]:
dr_predictions.columns = ['Number of Hours', 'Predicted Hours']

In [ ]:
dr_predictions['Hours per Provider'] = dr_predictions['Predicted Hours'] / num_dr['Number_Providers']

In [ ]:
avg_hours_per_dr = dr_predictions['Hours per Provider'].mean()

In [ ]:
dr_predictions['Predicted # of Providers'] = np.ceil(dr_predictions['Predicted Hours'] / avg_hours_per_dr)

In [ ]:
# save to csv file
dr_predictions.to_csv('./data/dr_arimax_predictions.csv')

#### RN/PAs

In [ ]:
RNPA_forecast, RNPA_predictions = get_ARIMAX_predictions(data=RNPA_hours, order=(4,1,0),\
        start=start_date, end=end_pred, exog=num_RNPA, typ='levels')

In [ ]:
RNPA_predictions.columns = ['Number of Hours', 'Predicted Hours']

In [ ]:
RNPA_predictions['Hours per Provider'] = RNPA_predictions['Predicted Hours'] / num_RNPA['Number_Providers']

In [ ]:
avg_hours_per_RNPA = RNPA_predictions['Hours per Provider'].mean()

In [ ]:
RNPA_predictions['Predicted # of Providers'] = np.ceil(RNPA_predictions['Predicted Hours'] / avg_hours_per_RNPA)

In [ ]:
# save to csv file
RNPA_predictions.to_csv('./data/RNPA_arimax_predictions.csv')

#### therapists

In [ ]:
ther_forecast, ther_predictions = get_ARIMAX_predictions(data=therapist_hours, order=(5,1,0),\
        start=start_date, end=end_pred, exog=num_therapists, typ='levels')

In [ ]:
ther_predictions.columns = ['Number of Hours', 'Predicted Hours']

In [ ]:
ther_predictions['Hours per Provider'] = ther_predictions['Predicted Hours'] / num_therapists['Number_Providers']

In [ ]:
avg_hours_per_therapist = ther_predictions['Hours per Provider'].mean()

In [ ]:
ther_predictions['Predicted # of Providers'] = np.ceil(ther_predictions['Predicted Hours'] / avg_hours_per_therapist)

In [ ]:
# save to csv file
ther_predictions.to_csv('./data/therapist_arimax_predictions.csv')

#### import hours per provider dataframes and split into training and test data

In [ ]:
dr_df = pd.read_csv('./data/doctors_hours_per_provider.csv', index_col=0)
RNPA_df = pd.read_csv('./data/RNPA_hours_per_provider.csv', index_col=0)
ther_df = pd.read_csv('./data/therapists_hours_per_provider.csv', index_col=0)

In [ ]:
train_start = '2015-01-12'
train_end = '2018-02-26'
test_start = '2018-03-05'

In [ ]:
dr_train = dr_df.loc[train_start:train_end]
dr_test = dr_df.loc[test_start:]

In [ ]:
RNPA_train = RNPA_df.loc[train_start:train_end]
RNPA_test = RNPA_df.loc[test_start:]

In [ ]:
ther_train = ther_df.loc[train_start:train_end]
ther_test = ther_df.loc[test_start:]

#### get MSE (train and test data) for each category to compare with other models

In [ ]:
train_dfs = [dr_train, RNPA_train, ther_train]
test_dfs = [dr_test, RNPA_test, ther_test]
orders = [(5,1,3), (4,1,0), (6,1,0)]
endog_col = 'Hours'
exog_col = 'Number_Providers'
names = ['Doctors', 'RN/PAs', 'Therapists']
test_start='2018-03-05'
train_start = '2015-01-12'
train_end = '2018-02-26'

In [ ]:
# get train/test predictions and MSE for all 3 categories for MA=0
train_mse = []
test_mse = []
category = []

for i in range(len(train_dfs)):
    forecast, full_df = get_ARIMAX_train_test_predictions(training_data=train_dfs[i][endog_col],\
            test_data=test_dfs[i][endog_col], order=orders[i], start='2015-01-19',\
            end='2018-04-30', exog=train_dfs[i][exog_col], typ='levels')
    train_error, test_error = get_ARIMAX_train_test_MSE(full_df, data_col='data', pred_col='forecast',\
                        train_end='2018-02-26', test_start='2018-03-05', data_name=names[i])   
    train_mse.append('training set: {}'.format(train_error))
    test_mse.append('test set: {}'.format(test_error))
    category.append('{} MSE'.format(names[i]))
mse = zip(train_mse, test_mse)
print(list(zip(category,mse)))